# TIMESERIES ANALYSIS

# Project Description

This project focuses on the application of time series regression analysis to forecast sales for Corporation Favorita, a prominent grocery retailer based in Ecuador.

The primary objective is to develop a robust model capable of accurately forecasting future sales by leveraging the extensive time series data of thousands of products sold across various Favorita locations. The resulting forecasts will provide valuable insights to the store's management, enabling them to formulate effective inventory and sales plans.

# Business Understanding

Sales is the primary parameter for measuring the success of any business, and Corporation Favorita is no exception. The business has the data of its sales from 2013 to 2017 across its stores located in different cities in Ecuador. During this time period, there have been holiday events, increase in oil price and an earthquake in Ecuador. The business will like to know the impact of these occurrences on its sales, as well as the sales performance of its stores across different store types, clusters, cities and states.

Furthermore, the business will like to know if there is a trend in the sales data, as well as the occurrence of seasonality to help identify the buying patterns of customers and prepare adequately for expected peak sales periods. The impact of promotions on sales is another key factor the business wants to derive insigts on.

This project is aimed at providing these business insights by performing historical analysis using the available time-stamped historical data, establishing scientific hypothesis, and building an optimal time-series regression model to predict future sales. This prediction will ensure strategic decision-making in the future. By delving into the data, my goal is to optimize operations and ultimately drive sales growth for Corporation Favorita, support the management team in extracting meaningful insights from their vast data, and help drive data-informed decision making.

# Hypothesis

Null Hypothesis: Sales are not affected by promotion, oil prices and holidays.

Alternate Hypothesis: Sales are affected by promotion, oil prices and holidays.

# Analytical Questions

1. Is the train dataset complete (has all the required dates)?

2. Which dates have the lowest and highest sales for each year?

3. Did the earthquake impact sales?

4. Are certain groups of stores selling more products? (Cluster, city, state, type)

5. Are sales affected by promotions, oil prices and holidays?

6. What analysis can we get from the date and its extractable features?

7. What is the difference between RMSLE, RMSE, MSE (or why is the MAE greater than all of them?)

8. What is the total sales made each year by the corporation?

In [16]:
# Installations

# !pip install pyodbc
# !pip install python-dotenv
# !pip install sqlalchemy
# !pip install lightgbm
# !pip install xgboost
# !pip install catboost
# !pip install pmdarima

In [17]:
# Importing the needed packages
import pandas as pd
import numpy as np
import matplotlib.dates as ndates

# Libraries for visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Libraries to create connection string to SQL server
import pyodbc
from dotenv import dotenv_values

# Library for imputing missing values
from sklearn.impute import SimpleImputer

# Library for seasonal decomposition
from statsmodels.tsa.seasonal import seasonal_decompose

# Library for checking stationarity
from statsmodels.tsa.stattools import adfuller

# Library for feature scaling
from sklearn.preprocessing import StandardScaler

# Library for feature encoding
from sklearn.preprocessing import OneHotEncoder

# Libraries for modelling
from sklearn.linear_model import LinearRegression
import xgboost as xgb
from catboost import CatBoostRegressor
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.arima.model import ARIMA
from pmdarima import auto_arima
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Libraries for calculating evaluation metrics
from sklearn.metrics import mean_squared_error, mean_squared_log_error

# Library to make series stationary
from scipy.stats import boxcox

# Library for working with operating system
import os

# Library to handle warnings
import warnings
warnings.filterwarnings('ignore')

In [18]:
# Set Matplotlib defaults
plt.style.use("seaborn-v0_8")
plt.rc(
    "figure",
    autolayout=True,
    figsize=(11, 4),
    titlesize=18,
    titleweight='bold',
)
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=16,
    titlepad=10,
)
plot_params = dict(
    color="0.75",
    style=".-",
    markeredgecolor="0.25",
    markerfacecolor="0.25",
    legend=False,
)
%config InlineBackend.figure_format = 'retina'

# Data Understanding

### Accessing and loading the datasets

The first dataset was collected from a SQL database by first passing a connection string using the pyodbc library. Afterwards a SQL query was used to obtain the dataset. This is as shown below.

In [19]:
# Load the environment variable in the .env file into a dictionary

environment_variables = dotenv_values('.env')

# Get the values for the credentials you set in the .env file
server = environment_variables.get('SERVER')
database = environment_variables.get('DATABASE')
username = environment_variables.get('USERNAME')
password = environment_variables.get('PASSWORD')

# The connection string is an f string that includes all the variable above to establish a connection to the server.
connection_string = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}'

In [20]:
# Use the connect method of the pyodbc library to pass in the connection string.
# Check your internet connection if it takes more time than necessary.

connection = pyodbc.connect(connection_string)

# Get the oil dataset using the SQL query shown below
query1 = 'Select * from dbo.oil'
oil = pd.read_sql(query1, connection)

# Get the holiday dataset using the SQL query shown below
query2 = 'Select * from dbo.holidays_events'
holiday = pd.read_sql(query2, connection)

# Get the stores dataset using the SQL query shown below
query3 = 'Select * from dbo.stores'
stores = pd.read_sql(query3, connection)

# Save the datasets
oil.to_csv(r'oil.csv')
holiday.to_csv(r'holiday.csv')
stores.to_csv(r'stores.csv')

In [21]:
# Close connection

connection.close

In [ ]:
# Loading the other datasets

transactions = pd.read_csv('transactions.csv')
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample_submission = pd.read_csv('sample_submission.csv')

### EDA

In [ ]:
# Evaluating the first five rows of the holiday dataset

holiday.head()

In [ ]:
# Evaluating the first five rows of the oil dataset

oil.head()

In [ ]:
# Visualize the trend of oil prices

oil_df = oil.set_index('date')
oil_df.plot()

In [ ]:
# Evaluating the first five rows of the stores dataset

stores.head()

In [ ]:
# Evaluating the first five rows of the transactions dataset

transactions.head()

In [ ]:
# Evaluating the first five rows of the train dataset

train.head()

In [ ]:
# Evaluating the first five rows of the test dataset

test.head()

In [ ]:
# Evaluating the first five rows of the sample_submission dataset

sample_submission.head()

In [ ]:
# Create a dictionary of the datasets

data = {'holiday': holiday, 'oil': oil, 'stores': stores, 'transactions': transactions, 'train': train, 'test': test, 'sample_submission': sample_submission}

In [ ]:
# Check the datatypes and presence of missing values in each of the datasets
# Use '\033[1mtext\033[0m' to make text bold

for df, dataset in data.items():
    print(f'\033[1mFor {df} dataset\033[0m:')
    dataset.info()
    print('_'*45)
    print('\n')

In [ ]:
# Check the shape, and the presence of missing values and duplicates in each of the datasets
# Use '\033[1mtext\033[0m' to make text bold

for df, dataset in data.items():
    print(f'\033[1mFor {df} dataset\033[0m')
    print(f'Shape: {dataset.shape}')
    print(f'Missing values = {dataset.isna().sum().sum()}')
    print(f'Duplicates = {dataset.duplicated().sum()}')
    print('_'*30)
    print('\n')

# Problems Identified

The datasets are seperate, and need to be merged together for better analysis.

The oil dataset has 43 missing values on the 'dcoilwtico' column which should be filled.

Each of the 'date' columns have an object datatype instead of a datetime datatype.

# Data Preparation

The problems identified with the datasets will be handled to prepare the data for analysis and modelling.

### Merge the datasets based on common columns

In [ ]:
# Merge transactions dataset to train on 'date' and 'store_nbr' columns
df1 = pd.merge(train, transactions, on=['date', 'store_nbr'], how='left')

# Merge holiday dataset to df1 on 'date' column
df2 = pd.merge(df1, holiday, on='date', how='left')

# Merge oil dataset to df2 on 'date' column
df3 = pd.merge(df2, oil, on='date', how='left')
df3

# Merge store dataset to df3 on 'store_nbr' column
df4 = pd.merge(df3, stores, on='store_nbr', how='left')

df4.head()

In [ ]:
# Check for duplicates after merging the datasets

df4.duplicated().sum()

In [ ]:
# Create a copy of the dataset

df = df4.copy()

In [ ]:
# Evaluating 'type_x' column on df4

df['type_x'].unique()

In [ ]:
# Evaluating 'type_y' column on df4

df['type_y'].unique()

As can be seen in the merged dataset, the column named type_x is the type column of the holiday dataset, the column named dcoilwtico represents the oil price in the oil dataset, while the column named type_y is the type column of the store dataset. These columns will be renamed for easy identification.

In [ ]:
# Rename 'type_x', 'dcoilwtico' and type_y' to 'holiday_type', 'oil_price' and 'store_type' respectively

df = df.rename(columns={'type_x': 'holiday_type', 'dcoilwtico': 'oil_price', 'type_y': 'store_type'})
df.head()

### Filling missing values

In [ ]:
# Check for missing values after merging the datasets

df.isna().sum()

The missing values in the transactions column will be filled with 0 because it represents the absence of transactions on those days.

In [ ]:
# Fill the missing values in the transactions column with 0

df['transactions'].fillna(0, inplace=True)
df.isna().sum()

For holiday_type, locale, locale_name, description and transferred columns, there are equal number of missing values. This is because these columns are from the holiday dataset, and they represent the days where there were no holidays. These empty cells will be filled with 'No holiday' for easy identification.

In [ ]:
# Fill the missing values in type_x, locale, locale_name, description and transferred columns with 'No holiday'
columns_to_fill = ['holiday_type', 'locale', 'locale_name', 'description', 'transferred']
for column in columns_to_fill:
        df[column].fillna('No holiday', inplace=True)

# Confirm that there are no more missing values in these columns
df.isna().sum()

In [ ]:
# Visualize the trend of oil prices before filling the missing values

df['oil_price'].plot()

In [ ]:
# Fill the missing values in the oil prices using backward fill to ensure continuity in the trend
df['oil_price'].fillna(method='bfill', inplace=True)

# Confirm that there are no more missing values
df.isna().sum()

In [ ]:
# Visualize the trend of oil prices after filling the missing values

df['oil_price'].plot()

### Change the datatype of the 'date' column from object to datetime

In [ ]:
# Change the datatype of the date column to datetime
df['date'] = pd.to_datetime(df['date'])

df.info()

In [ ]:
# Create new features from the 'date' column using pandas' powerful time-based indexing

df['year'] = df.date.dt.year
df['month'] = df.date.dt.month
df['dayofmonth'] = df.date.dt.day
df['dayofweek'] = df.date.dt.dayofweek
df['dayname'] = df.date.dt.strftime('%A')

df.head()

In [ ]:
# Set the 'date' column as index

df = df.set_index('date')
df.head()

In [ ]:
# Rename df to train_merged

train_merged = df.copy()

In [ ]:
# Display random sample of 8 rows

train_merged.sample(8, random_state = 42)

In [ ]:
# Visualize the 'transactions' column

plt.figure(figsize=(15,10))
ax = train_merged['transactions'].plot(linewidth=0.5)
ax.set_ylabel('Transactions')
ax.set_xlabel('Year')

The plot above reveals that transactions are always highest at the end of each year. This reveals seasonality in the dataset.

In [ ]:
# Perform seasonal decomposition

result = seasonal_decompose(train_merged['sales'], model='additive', period=365)
result.plot()
plt.show()

In [ ]:
# # Check for stationarity using adfuller

# result = adfuller(series, autolag='AIC')

# print(f'ADF Statistics: {result[0]}')
# print(f'p-value: {result[1]}')
# print(f'Critical Values: {result[4]}')

# if result[1] > 0.05 :
#     print('Series is not stationary')
# else:
#     print('Series is stationary')

In [ ]:
# Create a lag plot

pd.plotting.lag_plot(train_merged['transactions'], lag=1)

In [ ]:
# Create a boxplot of the 'transactions' column grouped by 'locale'
sns.boxplot(x='transactions', y='locale', data=train_merged)

# Show the plot
plt.show()

In [ ]:
# Create a histogram of the 'transactions' column
train_merged.transactions.hist()

# Add labels to the x-axis, y-axis, and title
plt.xlabel('Transactions', fontsize=16)
plt.ylabel('Frequency', fontsize=16)
plt.title('Histogram of Transactions', fontsize=20)

# Show the plot
plt.show()

In [ ]:
# Visualize the sales trend of the dataset

plt.figure(figsize=(15,8))
sns.lineplot(x='date', y='sales', data=train_merged)
plt.title('Sales Trend Of The Dataset')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.show()

In [ ]:
# Generate summary statistics for numerical columns in train_data DataFrame

train_merged.describe()

In [ ]:
# Create a DataFrame of train_data with numerical columns only
train_merged_num = train_merged.select_dtypes(include=['float64', 'int64'])

# Calculate the correlation matrix of the numerical columns
corr_matrix = train_merged_num.corr()

# Visualizing the correlation matrix with a heatmap
sns.heatmap(corr_matrix, annot=True)

# Save the chart as an image file
# plt.savefig('Correlation of the numerical columns of the train dataset.png')

# Hypothesis Testing

# Answering Analytical Questions

### Questions

1. Is the train dataset complete (has all the required dates)?

2. Which dates have the lowest and highest sales for each year?

3. Did the earthquake impact sales?

4. Are certain groups of stores selling more products? (Cluster, city, state, type)

5. Are sales affected by promotions, oil prices and holidays?

6. What analysis can we get from the date and its extractable features?

7. What is the difference between RMSLE, RMSE, MSE (or why is the MAE greater than all of them?)

8. What is the total sales made each year by the corporation?

### Question 1

Is the train dataset complete (has all the required dates)?

In [ ]:
# Check the range of the date column
dates_range = pd.date_range(start=train_merged.index.min(), end=train_merged.index.max())

# Check for missing dates in the dataset
missing_dates = set(dates_range.date) - set(train_merged.index.unique())

# Create a new dataframe with the dates_missing data
missing_dates_df = pd.DataFrame(missing_dates)
missing_dates_df.head()

The dataset has some missing dates. This means that it is not complete.

### Question 2

Which dates have the lowest and highest sales for each year?

### Question 3

Did the earthquake impact sales?

In [ ]:
# Define time period before and after the earthquake
pre_earthquake_start_date = '2016-04-01'
pre_earthquake_end_date = '2016-04-15'
post_earthquake_start_date = '2016-04-17'
post_earthquake_end_date = '2016-04-30'

# Filter the sales data before and after the earthquake
pre_earthquake_sales = train_merged[(train_merged.index >= pre_earthquake_start_date) & (train_merged.index <= pre_earthquake_end_date)]
post_earthquake_sales = train_merged[(train_merged.index >= post_earthquake_start_date) & (train_merged.index <= post_earthquake_end_date)]

# Calculate the total sales before and after the earthquake
pre_earthquake_total_sales = pre_earthquake_sales['sales'].sum()
post_earthquake_total_sales = post_earthquake_sales['sales'].sum()

# Visualize the sales data before and after the sales data
labels = ['Pre-Earthquake', 'Post-Earthquake']
total_sales = [pre_earthquake_total_sales, post_earthquake_total_sales]
plt.bar(labels, total_sales)
plt.xlabel('Before And After Earthquake')
plt.ylabel('Total Sales')
plt.title('Impact Of Earthquake On Sales')
plt.show()

The plot above shows that there was a slight increase in sales after the earthquake. This means that the earthquake did not affect the sales.

### Question 4

Are certain groups of stores selling more products? (Cluster, city, state, type)

In [ ]:
# Compare the sales in the different store clusters

plt.figure(figsize=(10,5))
sns.barplot(x='cluster', y='sales', data=train_merged)
plt.title('Sales In Different Store Clusters')
plt.xlabel('Store Clusters')
plt.ylabel('Sales')
plt.show()

The plot above shows that stores in cluster 5 are making more sales than the stores in other clusters.

In [ ]:
# Compare the sales in different cities

plt.figure(figsize=(10,5))
sns.barplot(x='city', y='sales', data=train_merged)
plt.title('Sales In Different Cities')
plt.xlabel('Cities')
plt.ylabel('Sales')
plt.xticks(rotation = 45)
plt.show()

The plot above shows that stores in Quito are making more sales than stores in other cities.

In [ ]:
# Compare the sales in different states

plt.figure(figsize=(10,5))
sns.barplot(x='state', y='sales', data=train_merged)
plt.title('Sales In Different States')
plt.xlabel('States')
plt.ylabel('Sales')
plt.xticks(rotation = 80)
plt.show()

The plot above shows that stores in Pichincha are making more sales than stores in other states.

In [ ]:
# Compare the sales in different store types

store_types = ['A', 'B', 'C', 'D', 'E']

plt.figure(figsize=(10,5))
sns.barplot(x='store_type', y='sales', data=train_merged, order=store_types)
plt.title('Sales In Different Store Types')
plt.xlabel('Store Type')
plt.ylabel('Sales')
plt.show()

The plot above shows that stores in Store type A are making more sales than stores in other store types.

### Question 5

Are sales affected by promotions, oil prices and holidays?

In [ ]:
# Identify families with at least one item on promotion and families with no item on promotion
train_merged['onpromotion'] = train_merged['onpromotion'].apply(lambda x: 'No Promotion' if x == 0 else 'Promotion')

# Group by promotion and sum the sales
x = train_merged.groupby(['onpromotion'], as_index=False).agg({'sales':'sum'})

# Plot the sales of promotion and non_promotion families)
plt.figure(figsize=(8,5))
plt.bar(x.onpromotion, x.sales)
plt.title('Impact Of Promotion On Sales')
plt.xticks((0,1))
plt.xlabel('Promotion Status')
plt.ylabel('Sales')

The plot above shows that sales are affected by promotion. Product families with items on promotion are being sold more than product families with no item on promotion.

In [ ]:
# Compare the sales with different oil prices

plt.figure(figsize=(10,5))
sns.scatterplot(x='oil_price', y='sales', data=train_merged)
plt.title('Sales With Different Oil Prices')
plt.xlabel('Oil Prices')
plt.ylabel('Sales')
plt.show()

The plot above shows that sales are affected by the oil prices. As shown, there are more number of sales and more volume of sales at lower oil prices than at higher oil prices.

In [ ]:
# Visualize the holiday events

plt.figure(figsize=(10,5))
sns.countplot(x='holiday_type', data=train_merged)
plt.title('Count of Holiday Events By Type')
plt.xlabel('Holiday Events')
plt.ylabel('Count')
plt.show()

In [ ]:
# Compare the sales on holidays with sales on non-holidays

plt.figure(figsize=(10,5))
sns.barplot(x='holiday_type', y='sales', data=train_merged)
plt.title('Sales On Holidays Vs Non-holidays')
plt.xlabel('Is Holidays')
plt.ylabel('Sales')
plt.show()

There are more sales during holidays than when there are no holidays. This is because more people are free to go out for shopping during holidays than when there is no holiday and they have to be at work or school.

In [ ]:
# Compare the sales on holidays with sales on non-holidays for the different store types

plt.figure(figsize=(10,5))
sns.barplot(x='store_type', hue='holiday_type', y='sales', data=train_merged, ci=None, order=store_types)
plt.title('Sales On Holidays Vs Non-holidays For Each Store Type')
plt.xlabel('Is Holidays')
plt.ylabel('Sales')
plt.legend(title='Holiday Type')
plt.show()

There is no signicant impact of holidays on the sales made.

### Question 6

What analysis can we get from the date and its extractable features?

In [ ]:
# From the date and its extractable features, the average sales on different days of the week can be analyzed

days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

plt.figure(figsize=(10,5))
sns.barplot(x='dayname', y='sales', data=train_merged, order=days)
plt.title('Sales On Different Days Of The Week')
plt.xlabel('Days Of The Week')
plt.ylabel('Sales')
plt.show()

From the date and its extractable features, we can observe the sales according to the days of the week. As shown above, there are more sales during the weekends, with peak sales on Sundays.

### Question 7

What is the difference between RMSLE, RMSE, MSE (or why is the MAE greater than all of them?)

### Question 8

What is the total sales made each year by the corporation?

In [ ]:
# Group by year and sum of sales for each year

sales_per_year = train_merged.groupby(['year'], as_index=False).agg({'sales':'sum'})
sales_per_year

In [ ]:
# Plot the sales made each year

plt.bar(sales_per_year.year, sales_per_year.sales)
plt.title('Sales Per Year',fontsize=14)
plt.ylabel('Sales',fontsize=14)
plt.xlabel('Year',fontsize=14)
plt.show()

From the data provided, Corporation Favorita made it's lowest sales in 2013 and its highest sales in 2016.

# Feature Engineering

In [ ]:
# Make a copy of the train_merged dataset on which to perform feature engineering

train_data = train_merged.copy()

In [ ]:
# Reset the index back to a normal column

train_data = train_data.reset_index()
train_data.head()

### Drop Unnecessary columns

In [ ]:
# Display all the columns of the dataset

train_data.columns

In [ ]:
# Display random sample of 5 rows to see the contents of the columns

train_data.head()

In [ ]:
# Drop some columns and display the dataset

columns_to_drop = ['id', 'store_nbr', 'locale', 'locale_name', 'description', 'transferred', 'city', 'store_type', 'cluster', 'month', 'dayofmonth', 'dayofweek', 'dayname']

train_data = train_data.drop(columns_to_drop, axis=1)
train_data.head()

### Data Splitting

The dataset will be splitted to training and validation sets using the time-based split. This will be done based on the year.

In [ ]:
# List the years in the dataset

train_data['year'].unique()

In [ ]:
# Establish the years for the training set and validation set
train_years = [2013, 2014, 2015, 2016]
val_year = [2017]

# Obtain the training set and validation set
train_set = train_data.loc[train_data['year'].isin(train_years)]
val_set = train_data.loc[train_data['year'].isin(val_year)]

# Print the shape of the training set and validation set
train_set.shape, val_set.shape

In [ ]:
# Check the datatypes of the columns of the training set

train_set.info()

In [ ]:
# Obtain the categorical columns to encode and numerical columns to scale
cat_columns_to_encode = ['family', 'onpromotion', 'holiday_type', 'state']
num_columns_to_scale = ['transactions', 'oil_price']

# Create seperate DataFrames for categorical columns and numerical columns of training set
train_set_cat_df = train_set[cat_columns_to_encode]
train_set_num_df = train_set[num_columns_to_scale]

# Create seperate DataFrames for categorical columns and numerical columns of validation set
val_set_cat_df = val_set[cat_columns_to_encode]
val_set_num_df = val_set[num_columns_to_scale]

### Feature Encoding

In [ ]:
# Create an encoder object using OneHotEncoder.
# Set sparse=False for dense output and drop='first' to avoid multicollinearity

encoder = OneHotEncoder(sparse=False, drop='first')

In [ ]:
# Use OneHotEncoder to encode the categorical columns of the training set
encoder.fit(train_set_cat_df)
train_set_cat_encoded = encoder.transform(train_set_cat_df).tolist()
train_set_cat_encoded_df = pd.DataFrame(train_set_cat_encoded, columns=encoder.get_feature_names_out())

# View the encoded columns of the training set
train_set_cat_encoded_df.head()

In [ ]:
# Use OneHotEncoder to encode the categorical columns of the training set
encoder.fit(val_set_cat_df)
val_set_cat_encoded = encoder.transform(val_set_cat_df).tolist()
val_set_cat_encoded_df = pd.DataFrame(val_set_cat_encoded, columns=encoder.get_feature_names_out())

# View the encoded columns of the validation set
val_set_cat_encoded_df.head()

### Feature Scaling

In [ ]:
# Create a scaler object using StandardScaler

scaler = StandardScaler()

In [ ]:
# Use StandardScaler to scale the numerical columns of the training set
scaler.fit(train_set_num_df)
train_set_num_scaled = scaler.transform(train_set_num_df).tolist()
train_set_num_scaled_df = pd.DataFrame(train_set_num_scaled, columns=scaler.get_feature_names_out())

# View the scaled columns of the training set
train_set_num_scaled_df.head()

In [ ]:
# Use StandardScaler to scale the numerical columns of the validation set
scaler.fit(val_set_num_df)
val_set_num_scaled = scaler.transform(val_set_num_df).tolist()
val_set_num_scaled_df = pd.DataFrame(val_set_num_scaled, columns=scaler.get_feature_names_out())

# View the scaled columns of the validation set
val_set_num_scaled_df.head()

In [ ]:
# Combine the encoded and scaled DataFrames with the date and sales columns of train_set and val_set
# to get the final training and validation sets

train_final = pd.concat([train_set_cat_encoded_df, train_set_num_scaled_df, train_set[['date', 'sales']]], axis=1)
val_final = pd.concat([val_set_cat_encoded_df, val_set_num_scaled_df, val_set[['date', 'sales']]], axis=1)

In [ ]:
# Concatenate the encoded X_train DataFrame and the scaled X_train DataFrame to have the unbalanced X_train DataFrame

X_train = pd.concat([X_train_cat_encoded_df, X_train_num_scaled_df.set_axis(X_train_cat_encoded_df.index)], axis=1)
X_train.head()

In [ ]:
# Concatenate the encoded X_val DataFrame and the scaled X_val DataFrame to have the ready X_val DataFrame

X_val = pd.concat([X_val_cat_encoded_df, X_val_num_scaled_df.set_axis(X_val_cat_encoded_df.index)], axis=1)
X_val.head()

In [ ]:
# series = train_merged.loc[:, 'transactions'].values
# series

# Modelling

The following models will be built and evaluated:
1. Linear Regression

2. XGBoost

3. CatBoost

4. ARIMA

5. SARIMA

In [ ]:

# def evaluate_time_series_models(data, date_column, target_column):
#     # Assuming you have your time series data in a DataFrame 'data'
#     # Replace 'date_column' with the column containing date/time and 'target_column' with the target variable column name
#     data[date_column] = pd.to_datetime(data[date_column])
#     data.set_index(date_column, inplace=True)
#     target = data[target_column]

#     # Splitting the data into training and testing sets (you can adjust the split ratio as per your requirement)
#     train_size = int(len(target) * 0.8)
#     train, test = target[:train_size], target[train_size:]

# Initialize model names and corresponding models
model_names = ['Linear Regression', 'XGBoost', 'CatBoost', 'ARIMA', 'SARIMA']
models = [LinearRegression(), xgb.XGBRegressor(), CatBoostRegressor(), ARIMA(train, order=(1, 0, 0)),
          SARIMAX(train, order=(1, 0, 0), seasonal_order=(1, 0, 0, 12))]

# Create a DataFrame to store the evaluation results
results = pd.DataFrame(columns=['Model', 'MSE', 'RMSE', 'RMSLE', 'MSLE'])

for model_name, model in zip(model_names, models):
    if model_name in ['ARIMA', 'SARIMA']:
        # Fit the ARIMA or SARIMA model
        model_fit = model.fit()

        # Make predictions
        model_preds = model_fit.forecast(steps=len(test))
    else:
        # Fit other models
        model.fit(np.arange(len(train)).reshape(-1, 1), train)

        # Make predictions
        model_preds = model.predict(np.arange(len(test)).reshape(-1, 1))

    # Calculate evaluation metrics
    mse = mean_squared_error(test, model_preds)
    rmse = np.sqrt(mse)
    rmsle = np.sqrt(mean_squared_log_error(test, model_preds))
    msle = mean_squared_log_error(test, model_preds)

    # Append the results to the DataFrame
    results = results.append({'Model': model_name, 'MSE': mse, 'RMSE': rmse, 'RMSLE': rmsle, 'MSLE': msle},
                             ignore_index=True)

return results

# Assuming you have your time series data in a DataFrame 'data' with columns 'Date' and 'Target'
# Adjust the column names accordingly
results = evaluate_time_series_models(train_merged, date_column='Date', target_column='y_val')
results

In [ ]:
# Initialize the model
catboost_model = CatBoostRegressor()

# Fit the model
catboost_model.fit(np.arange(len(X_train)).reshape(-1, 1), X_train)

# Make predictions
catboost_preds = catboost_model.predict(np.arange(len(X_val)).reshape(-1, 1))

# Calculate evaluation metrics
mse_catboost = mean_squared_error(X_val, catboost_preds)
rmse_catboost = np.sqrt(mse_catboost)
rmsle_catboost = np.sqrt(mean_squared_log_error(X_val, catboost_preds))
msle_catboost = mean_squared_log_error(X_val, catboost_preds)

### AR

In [ ]:
# Plot the pacf and acf

pacf = plot_pacf(data['sales'], lags=10)
acf = plot_acf(data['sales'], lags=10)

In [ ]:
model = AutoReg(data, lags=5).fit()
model.summary

In [ ]:
AR_pred = model.predict(start=len(train), end=len(train)+len(test)-1, dynamic=False)

In [ ]:
plt.figure(figsize=(12,4))
plt.plot(train['sales'], label='Train')
plt.plot(val['sales'], label='Validation')
plt.plot(AR_pred, label='AR Forecast')
plt.title('AutoRegressive (AR) Method')
plt.legend(loc='best')
plt.show()

In [ ]:
# Create an empty DataFrame to store the results
results = pd.DataFrame(columns=['Model', 'MSE', 'RMSE', 'RMSLE', 'MSLE'])

# Calculate evaluation metrics
mse = mean_squared_error(val, AR_pred)
msle = mean_squared_log_error(val, AR_pred)
rmse = np.sqrt(mse).round(2)
rmsle = np.sqrt(msle).round(2)

# Create a new row for the evaluation metrics of ARIMA model on the results DataFrame
ARIMA_results = pd.DataFrame([['AR', 'mse', 'rmse', 'rmsle', 'msle']], columns=['Model', 'MSE', 'RMSE', 'RMSLE', 'MSLE'])
results = results.append(ARIMA_results, ignore_index=True)

### ARIMA

In [ ]:
stepwise_fit = autoarima(train['sales'], trace=True, suppress_warnings=True)
stepwise_fit.summary()

In [ ]:
model = ARIMA(train, order=(1,1,1))
model_fit = model.fit()
model_fit.summary()

In [ ]:
# Make predictions on the validation set

ARIMA_pred = model.predict(start=len(train), end=len(train)+len(test)-1, typ='levels')
ARIMA_pred

In [ ]:
plt.figure(figsize=(12,4))
plt.plot(train['sales'], label='Train')
plt.plot(val['sales'], label='Validation')
plt.plot(ARIMA_pred, label='ARIMA Forecast')
plt.title('AutoRegressive Integrated Moving Average (ARIMA) method')
plt.legend(loc='best')
plt.show()

In [ ]:
# Calculate evaluation metrics
mse = mean_squared_error(val, ARIMA_pred)
msle = mean_squared_log_error(val, ARIMA_pred)
rmse = np.sqrt(mse).round(2)
rmsle = np.sqrt(msle).round(2) 

# Create a new row for the evaluation metrics of ARIMA model on the results DataFrame
ARIMA_results = pd.DataFrame([['ARIMA', 'mse', 'rmse', 'rmsle', 'msle']], columns=['Model', 'MSE', 'RMSE', 'RMSLE', 'MSLE'])
results = results.append(ARIMA_results, ignore_index=True)

### SARIMA

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

model = SARIMAX(train, order=(1,1,1), seasonal_order=(1,1,1,12))
model_fit = model.fit()
model_fit.summary()

In [ ]:
# Make predictions on the validation set

SARIMA_pred = model.predict(start=len(train), end=len(train)+len(test)-1, typ='levels')
SARIMA_pred

In [ ]:
plt.figure(figsize=(12,4))
plt.plot(train['sales'], label='Train')
plt.plot(val['sales'], label='Validation')
plt.plot(SARIMA_pred, label='SARIMA Forecast')
plt.title('Seasonal AutoRegressive Integrated Moving Average (SARIMA) method')
plt.legend(loc='best')
plt.show()

In [ ]:
# Calculate evaluation metrics
mse = mean_squared_error(val, SARIMA_pred)
msle = mean_squared_log_error(val, SARIMA_pred)
rmse = np.sqrt(mse).round(2)
rmsle = np.sqrt(msle).round(2) 

# Create a new row for the evaluation metrics of ARIMA model on the results DataFrame
SARIMA_results = pd.DataFrame([['SARIMA', 'mse', 'rmse', 'rmsle', 'msle']], columns=['Model', 'MSE', 'RMSE', 'RMSLE', 'MSLE'])
results = results.append(SARIMA_results, ignore_index=True)

# Make series stationary

We will use the boxcox method

In [ ]:
data_boxcox = pd.Series(boxcox(train['sales'], lbda=0), index=train.index)

plt.figure(figsize=(12,4))
plt.plot(data_boxcox, label='Before Boxcox Transformation')
plt.title('Before Boxcox Transformation')
plt.legend(loc='best')
plt.show()

In [ ]:
data_boxcox_diff = pd.Series(data_boxcox - data_boxcox.shift(), index=train.index)

plt.figure(figsize=(12,4))
plt.plot(data_boxcox_diff, label='After Boxcox Transformation')
plt.title('After Boxcox Transformation')
plt.legend(loc='best')
plt.show()

# Hyperparameter Tuning

# Prediction